In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics, manifold
from tqdm import tqdm
import gensim.downloader as gensim_api
import transformers
import os

In [2]:
df=pd.read_csv('output1.csv')
print(df["clean"])

0         rt mleew boy dats cold tyga dwn bad for cuffi...
1         rt urkindofbrand dawg rt sbaby life you ever ...
2         rt c g anderson viva based she look like a tr...
3         rt shenikaroberts the shit you hear about me ...
4         t madison x the shit just blows me claim you ...
                               ...                        
24628    u that nigga rt charmin harmon i have no idea ...
24629               u want a cookie tomorrow bitch lostb y
24630                                            u bitch u
24631                      ugh the bitch name was leather 
24632                                             ugly hoe
Name: clean, Length: 24633, dtype: object


In [3]:
nlp = gensim_api.load("glove-wiki-gigaword-300")

In [4]:
nlp.most_similar(["nigger"], topn=3)

[('slur', 0.5023007392883301),
 ('niggers', 0.5003143548965454),
 ('faggot', 0.4708784222602844)]

In [5]:
def get_similar_words(lst_words, top, nlp):
    lst_out = lst_words
    for tupla in nlp.most_similar(lst_words, topn=top):
        lst_out.append(tupla[0])
    return list(set(lst_out))

In [6]:
def similarWords(words, top, nlp):
    out = words
    for tupl in nlp.most_similar(words, topn=top):
        out.append(tupl[0])
    return list(set(out))

In [7]:
clusters={}
clusters['HATE']=get_similar_words(['hate'], top=30, nlp=nlp)

In [11]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('al', do_lower_case=True)
nlp = transformers.TFDistilBertModel.from_pretrained('al')

Some layers from the model checkpoint at al were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at al.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
def utils_bert_embedding(txt, tokenizer, nlp):
    '''
    Word embedding with Bert (equivalent to nlp["word"]).
    :parameter
        :param txt: string 
        :param tokenizer: transformers tokenizer
        :param nlp: transformers bert
    :return
        tensor sentences x words x vector (1x3x768) 
    '''
    # tokenize sentence to tokens (integers)
    idx = tokenizer.encode(txt)
    # convert to array of shape (1, num_words+2) - EOS and CLS added
    idx = np.array(idx)[None,:]
    # generate embeddings for each token - output is a tuple
    embedding = nlp(idx)
    # select first member of the tuple, remove first dimension which is 1 to get (num_words,embedding size 712)
    # exclude CLS and EOS tokens
    X = np.array(embedding[0][0][1:-1])
    return X

In [10]:
%%time
## create list of news vector
lst_mean_vecs = [utils_bert_embedding(txt, tokenizer, nlp).mean(0) for txt in tqdm(df["clean"])]

 22%|███████▋                           | 5446/24633 [20:44<1:05:43,  4.87it/s]

KeyboardInterrupt: 

In [ ]:
X = np.array(lst_mean_vecs)
X.shape